In [19]:
import pandas as pd

# 🔹 Dateipfade definieren
minergie_data_path = "plz_häufigkeit.csv"
apartment_data_path = "original_apartment_data_analytics_hs24_with_lat_lon.csv"
output_path = "precomputed_data.csv"

# 🔹 Daten laden
minergie_data = pd.read_csv(minergie_data_path)
apartments = pd.read_csv(apartment_data_path)

# 🔹 Datentypen korrigieren
minergie_data['PLZ'] = minergie_data['PLZ'].astype(str)
apartments['postalcode'] = apartments['postalcode'].astype(str)

# 🔹 Debugging: Anzahl Zeilen in jeder Datei
print(f"📊 Anzahl Zeilen - Apartments: {len(apartments)}")
print(f"📊 Anzahl Zeilen - Minergie: {len(minergie_data)}")

# 🔹 Durchschnittliche Werte pro PLZ berechnen
features_avg = apartments.groupby('postalcode')[['rooms', 'area', 'price']].mean().reset_index()
print(f"📊 Anzahl Zeilen nach Gruppierung (PLZ-Rooms, Area & Price): {len(features_avg)}")

# 🔹 Minergie-Daten mit Apartments verbinden
precomputed = features_avg.merge(minergie_data, left_on='postalcode', right_on='PLZ', how='left')
print(f"📊 Anzahl Zeilen nach Minergie-Merge: {len(precomputed)}")

# 🔹 Relevante Spalten behalten
precomputed = precomputed[['postalcode', 'rooms', 'area', 'price', 'Häufigkeit']]
precomputed.rename(columns={'Häufigkeit': 'minergie_anteil'}, inplace=True)

# 🔹 Fehlende Werte anzeigen
print("📌 Fehlende Werte vor Füllung:")
print(precomputed.isnull().sum())

# 🔹 Fehlende Werte mit 0 füllen
precomputed.fillna(0, inplace=True)

# 🔹 Debugging: Überprüfen, ob `postalcode` doppelt ist
print("📌 Anzahl einzigartiger PLZs:", precomputed['postalcode'].nunique())

# 🔹 Daten speichern
precomputed.to_csv(output_path, index=False)
print(f"✅ Neue Datei gespeichert unter: {output_path}")


📊 Anzahl Zeilen - Apartments: 819
📊 Anzahl Zeilen - Minergie: 261
📊 Anzahl Zeilen nach Gruppierung (PLZ-Rooms, Area & Price): 132
📊 Anzahl Zeilen nach Minergie-Merge: 132
📌 Fehlende Werte vor Füllung:
postalcode         0
rooms              0
area               0
price              0
minergie_anteil    0
dtype: int64
📌 Anzahl einzigartiger PLZs: 132
✅ Neue Datei gespeichert unter: precomputed_data.csv


In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd

# 🔹 Daten laden
data = pd.read_csv("precomputed_data.csv")

# 🔹 Features & Zielvariable definieren
features = ['lat', 'lon', 'area', 'rooms', 'minergie_anteil', 'tax_income', 
            'pop_dens', 'emp', 'price_per_sqm', 'avg_rent']
target = 'price'

X = data[features]
y = data[target]

# 🔹 Daten aufteilen & Modell trainieren
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 🔹 Modell speichern
joblib.dump(model, "price_prediction_model.pkl")


['price_prediction_model.pkl']

In [ ]:
import gradio as gr
import pandas as pd
import joblib
import numpy as np

# 🔹 **Dateipfade**
MODEL_PATH = "/workspaces/ai-applications-fs25/week2/test copy/price_prediction_model.pkl"
DATA_PATH = "/workspaces/ai-applications-fs25/week2/test copy/precomputed_data.csv"

# 🔹 **Modell & Daten einmalig laden**
print("📥 Lade Modell...")
model = joblib.load(MODEL_PATH)

print("📥 Lade vorverarbeitete Daten...")
precomputed = pd.read_csv(DATA_PATH)

# 🔹 **Sicherstellen, dass `town` eindeutige Werte hat**
precomputed = precomputed.groupby('town', as_index=False).mean()

# 🔹 **Daten als Dictionary für schnellen Zugriff**
precomputed_dict = precomputed.set_index('town').to_dict(orient='index')

# 🔹 **Dropdown-Liste mit Gemeindenamen**
gemeinde_options = sorted(precomputed['town'].unique())

beschreibung = """
🏡 **Wohnungspreis-Vorhersage mit Minergie & Steuerdaten**

🔹 Wähle **Wohnfläche (m²)**, **Zimmeranzahl** und **Gemeinde**  
🔹 Die App berechnet den erwarteten **Wohnungspreis** auf Basis von:
  - Wohnfläche
  - Anzahl der Zimmer
  - Standortdaten (Latitude, Longitude)
  - Steuer- & Minergie-Informationen

---

### 🔹 **Was ist Minergie?**
Minergie ist ein **Schweizer Baustandard** für **energieeffiziente Gebäude**.  
Ein Gebäude mit Minergie-Standard hat einen **geringeren Energieverbrauch**,  
eine **bessere Dämmung** und nutzt **erneuerbare Energien** wie Solarenergie oder Wärmepumpen.

---

📌 **Datenquelle:**
Die Minergie-Daten wurden durch **Web-Scraping** von  
[minergie.ch](https://www.minergie.ch/de/gebaeude/gebaeudeliste/?canton=zh&country=&zip_place=&street_nr=&gid=&typeofuse=&constructiontype=&year=&sortby=year_desc&numres=12&p=50)  
extrahiert, anschließend bereinigt und in das Modell integriert.
"""


# 🔹 **Vorhersagefunktion**
def predict_price(area, rooms, town):
    # Falls der Ort nicht in den Daten existiert
    if town not in precomputed_dict:
        return "🚨 Fehler: Keine Daten für diese Gemeinde verfügbar."

    # Werte aus dem Dictionary abrufen
    data = precomputed_dict[town]

    # 🔹 **Fehlende Features mit Standardwerten setzen**
    missing_features = ["pop_dens", "emp"]
    for feature in missing_features:
        data.setdefault(feature, 0)  # Falls Feature fehlt, auf 0 setzen

    # 🔹 **Modell-Eingaben vorbereiten (NumPy für schnellere Berechnungen)**
    input_data = np.array([[area, rooms, data['lat'], data['lon'], 
                            data['minergie_anteil'], data['tax_income'],
                            data['pop_dens'], data['emp'],  # Fehlende Features hinzugefügt
                            data['price_per_sqm'], data['avg_rent']]])

    # 🔹 **Modell-Vorhersage**
    predicted_price = model.predict(input_data)[0]

    return f"🏡 Erwarteter Wohnungspreis in {town}: **CHF {predicted_price:,.2f}**"

# 🔹 **Gradio Interface**
iface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Number(label="Wohnfläche (m²)", value=50),
        gr.Number(label="Zimmeranzahl", value=2),
        gr.Dropdown(choices=gemeinde_options, label="Gemeinde")
    ],
    outputs="text",
    title="🏡 Wohnungspreis-Vorhersage mit Minergie & Steuerdaten",
    description=beschreibung,
    theme="huggingface",
    live=False  # Verhindert unnötige Neuberechnungen bei Eingaben
)

# 🔹 **App starten**
iface.launch()


📥 Lade Modell...
📥 Lade vorverarbeitete Daten...


/home/codespace/.python/current/lib/python3.12/site-packages/gradio/blocks.py:1108: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-67d5eac4-29c0d50d0108daab0070dad7;d5757d1b-ac47-4698-a3c7-dcabdc011405)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but Gradient

In [5]:
import joblib

# Modell laden
model = joblib.load("price_prediction_model.pkl")

# Die Anzahl der erwarteten Features ausgeben
print("🚀 Erwartete Features des Modells:", model.feature_names_in_)
print("📊 Anzahl erwarteter Features:", model.n_features_in_)


🚀 Erwartete Features des Modells: ['lat' 'lon' 'area' 'rooms' 'minergie_anteil' 'tax_income' 'pop_dens'
 'emp' 'price_per_sqm' 'avg_rent']
📊 Anzahl erwarteter Features: 10
